In [1]:
import os
os.chdir("../")
with open('run/settings.py', 'r') as file:
    exec(file.read())

Random seed set to 333
using device:  cpu


In [2]:
# (customize) configs
overwrite = False
model_name = 'air_quality_kdd2' 

update_config(
    
    # Eval settings (clip)
    # ts2txt
    y_col = 'city_str',
    y_levels = ['The time series is about air quality in Beijing.', 'The time series is about air quality in London.'],
    y_pred_levels = ['The time series is about air quality in Beijing.', 'The time series is about air quality in London.'],
    # txt2ts
    txt2ts_y_cols = ['city_str', 'season_str'],
    
    
    # Data settings
    seq_length = 168,
    text_col = 'ts_description', #'ts_description',
    custom_target_cols = ['city_str', 'season_str', 'label'],
    
    
    # Model settings
    model_name = model_name,
    **{'3d': False},  # Add this line
    embedded_dim = 512,
    concat_embeddings = False,
    clip_mu = True,
    variational = True,
    train_type = 'joint', # or 'vae', 'clip'
    clip_target_type = 'by_target', # or 'by_label'
    
    # Train settings
    batch_size = 512,
    init_lr = 1e-5,
    patience = 100,
    alpha = 1/500,
    num_saves = 20,
    num_epochs = 500,
    
    # Text configuration
    text_config = text_config
)
config_dict = get_config_dict()


In [3]:
# change data preparation for a given experiment
with open('run/prepare_datasets/air_quality_kdd.py', 'r') as file:
    exec(file.read())
# prepare model inputs
with open('run/inputs.py', 'r') as file:
    exec(file.read())

After downsampling:
city_str
The time series is about air quality in Beijing.    1812
The time series is about air quality in London.     1236
Name: count, dtype: int64
After downsampling:
city_str
The time series is about air quality in Beijing.    453
The time series is about air quality in London.     310
Name: count, dtype: int64


final distribution of text prediction
text
The time series is about air quality in Beijing. The air quality is measured during winter.    580
The time series is about air quality in Beijing. The air quality is measured during spring.    499
The time series is about air quality in London. The air quality is measured during winter.     401
The time series is about air quality in Beijing. The air quality is measured during summer.    370
The time series is about air quality in Beijing. The air quality is measured during fall.      363
The time series is about air quality in London. The air quality is measured during spring.     329
The time series is about 

KeyboardInterrupt: 

In [ ]:
# customize encoder and decoder here. 
ts_encoder = MultiCNNEncoder(ts_dim = ts_f_dim.shape[1],
                             output_dim=config_dict['embedded_dim'],
                             kernel_sizes=[100, 50, 20, 5],
                             hidden_num_channel=16,
                             dropout=0)
ts_decoder = TransformerDecoder(ts_dim = ts_f_dim.shape[1], output_dim = config_dict['embedded_dim']+2, 
                 nhead = 8,
                 num_layers = 6,
                 dim_feedforward = 512,
                 dropout = 0.1)
text_encoder = TextEncoderMultiCNN(text_dim = tx_f_dim.shape[1],
                                   output_dim=config_dict['embedded_dim'],
                                   kernel_sizes=[500, 250, 100],  # Different context windows
                                   hidden_num_channel=16,
                                   dropout=0.0)
# overwrite = False
with open('run/model.py', 'r') as file:
    exec(file.read())


In [ ]:
# overwrite = False
with open('run/train.py', 'r') as file:
    exec(file.read())


In [ ]:
tid=0
# viz_generation_marginal(df_train, model, config_dict, tid=tid)
viz_generation_conditional(df_train, model, config_dict, tid=tid, sampling=True, b=100, ep=200)

In [ ]:
overwrite = False
# argument dictionary {y_col:conditions}
args0 = {'city_str': None,
        'season_str': None
        }

args1 = {'city_str': [('season_str', 'The air quality is measured during winter.')],
        'season_str': [('city_str', 'The time series is about air quality in Beijing.')]
        }

args2 = {'city_str': [('season_str', 'The air quality is measured during summer.')],
        'season_str': [('city_str', 'The time series is about air quality in London.')]
        }

args_ls = [args0, args1, args2]
filename = output_dir+'/df_dists_all.pt.gz'
if overwrite or (not os.path.exists(filename)):
    res_all = []
    for args in args_ls:
        dists_dict = {}
        for y_col in args.keys():
            print(y_col)
            df_dists = eval_ts_distances(df_test,
                                        model, 
                                        config_dict, 
                                        w = 0.8,
                                        y_col = y_col,
                                        conditions = args[y_col],
                                        b = 500)
            dists_dict[y_col] = df_dists

        # Save using PyTorch with gzip compression
        res = {'args':args, 'dists_dict':dists_dict}
        res_all.append(res)

    buffer = io.BytesIO()
    torch.save(res_all, buffer)
    with gzip.open(filename, 'wb') as f: f.write(buffer.getvalue())
else:
    # Load compressed file
    with gzip.open(filename, 'rb') as f:
        buffer = io.BytesIO(f.read())
        res_all = torch.load(buffer, map_location=device)



In [ ]:
# Define the base augmentation pairs
base_aug_dict = {'city_str': [('The time series is about air quality in Beijing.', 'The time series is about air quality in London.'), 
                            ('The time series is about air quality in London.', 'The time series is about air quality in Beijing.')],
                'season_str': [('The air quality is measured during winter.', 'The air quality is measured during summer.'),
                                ('The air quality is measured during summer.', 'The air quality is measured during winter.')]
                }
df_dists = pd.DataFrame()
for df in res_all[1]['dists_dict'].values():
    df_dists = pd.concat([df_dists, df], ignore_index=True)
# df_dists = res_all[0]['dists_dict']['segment1']
res_df = eng_dists_multiple(df_dists, base_aug_dict, metric = 'lcss')